In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

RAND_STATE = 32 # for reproducible shuffling
TT_RATIO = 0.25 # test/train

from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_barcelona = pd.read_csv('data_barcelona.csv')

In [3]:
data_barcelona.corr()

,Unnamed: 0,realSum,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
Unnamed: 0,1.000000,-0.002987,0.055059,-0.005826,-0.060303,-0.037052,-0.026683,0.069278,0.637230,0.372151,-0.494008,-0.560981,0.066976,-0.003790,-0.040555,-0.163928
realSum,-0.002987,1.000000,0.374672,-0.150752,0.261730,0.015031,-0.025710,0.381269,-0.036529,-0.015846,0.052208,0.027179,-0.031139,-0.451064,0.004984,0.016642
person_capacity,0.055059,0.374672,1.000000,-0.185463,0.365559,-0.054135,-0.111910,0.707166,-0.034859,-0.005281,0.040612,-0.010480,0.012257,-0.709682,-0.050022,-0.133572
multi,-0.005826,-0.150752,-0.185463,1.000000,-0.549697,0.006367,0.061617,-0.129066,0.038117,-0.011848,-0.015637,-0.014141,-0.029277,0.242047,0.062221,0.018713
biz,-0.060303,0.261730,0.365559,-0.549697,1.000000,-0.044428,-0.160341,0.220420,-0.169672,-0.089270,0.086974,0.113085,0.059097,-0.413508,-0.102203,-0.058932
cleanliness_rating,-0.037052,0.015031,-0.054135,0.006367,-0.044428,1.000000,0.732201,-0.021702,0.010978,0.001088,-0.002134,-0.013912,-0.024104,0.032923,0.246626,0.006560
guest_satisfaction_overall,-0.026683,-0.025710,-0.111910,0.061617,-0.160341,0.732201,1.000000,-0.051238,0.025465,0.038496,0.010253,-0.012203,-0.013458,0.107597,0.290455,0.023048
bedrooms,0.069278,0.381269,0.707166,-0.129066,0.220420,-0.021702,-0.051238,1.000000,0.032257,0.028804,-0.037039,-0.086784,-0.020350,-0.545034,0.019638,-0.118212
dist,0.637230,-0.036529,-0.034859,0.038117,-0.169672,0.010978,0.025465,0.032257,1.000000,0.431923,-0.650339,-0.826475,0.082276,0.057327,0.041585,-0.000266
metro_dist,0.372151,-0.015846,-0.005281,-0.011848,-0.089270,0.001088,0.038496,0.028804,0.431923,1.000000,-0.357704,-0.387814,0.044411,0.017559,0.069735,0.024707


In [4]:
data_barcelona = data_barcelona.drop(['Unnamed: 0', 'multi', 'cleanliness_rating', 'guest_satisfaction_overall', 'attr_index_norm', 'metro_dist'], axis=1)

In [5]:
x = data_barcelona.drop('realSum', axis=1)
y = data_barcelona.realSum

In [6]:
# test-train split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TT_RATIO, random_state=RAND_STATE)
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
x_train.describe()

,person_capacity,biz,bedrooms,dist,rest_index_norm,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
count,2124.000000,2124.000000,2124.000000,2124.000000,2124.000000,2124.000000,2124.000000,2124.000000,2124.000000
mean,2.619115,0.320151,1.164783,2.119449,19.300803,0.003296,0.805085,0.185970,0.451036
std,1.159219,0.466644,0.524629,1.366240,10.162773,0.057327,0.396229,0.389174,0.497714
min,2.000000,0.000000,0.000000,0.100000,3.518471,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,1.000000,1.087500,10.733975,0.000000,1.000000,0.000000,0.000000
50%,2.000000,0.000000,1.000000,1.750000,17.630011,0.000000,1.000000,0.000000,0.000000
75%,3.000000,1.000000,1.000000,2.985000,26.692060,0.000000,1.000000,0.000000,1.000000
max,6.000000,1.000000,6.000000,8.440000,100.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
x_train_const = sm.add_constant(x_train) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.297
Model:                            OLS   Adj. R-squared:                  0.294
Method:                 Least Squares   F-statistic:                     99.15
Date:                Mon, 06 Mar 2023   Prob (F-statistic):          1.05e-154
Time:                        16:58:29   Log-Likelihood:                -14778.
No. Observations:                2124   AIC:                         2.958e+04
Df Residuals:                    2114   BIC:                         2.963e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    228

In [8]:
sts = StandardScaler()

x_train_standarized = sts.fit_transform(x_train)
x_test_standarized = sts.transform(x_test)

x_train_standarized = pd.DataFrame(x_train_standarized)

In [9]:
x_train_const = sm.add_constant(x_train_standarized.to_numpy()) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test_standarized) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.297
Model:                            OLS   Adj. R-squared:                  0.294
Method:                 Least Squares   F-statistic:                     99.15
Date:                Mon, 06 Mar 2023   Prob (F-statistic):          1.05e-154
Time:                        16:58:29   Log-Likelihood:                -14778.
No. Observations:                2124   AIC:                         2.958e+04
Df Residuals:                    2114   BIC:                         2.963e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        289.0973      5.533     52.252      0.0

In [10]:
lm = LinearRegression()
model = lm.fit(x_train,y_train)
model.coef_
model.intercept_

228.3363765459448

In [11]:
model.score(x_train, y_train)

0.29682859888418434

In [12]:
y_pred = pd.DataFrame(model.predict(x_test),columns = ['realSum'] )      # model prediction
y_pred_train =  pd.DataFrame(model.predict(x_train),columns = ['realSum'])

In [13]:
mse(y_test,y_pred)

186900.70707014546

In [14]:
rmse = np.sqrt(mse(y_test,y_pred))
rmse

432.32014418732035

In [15]:
##prediction on the train set
mse(y_train,y_pred_train)

64711.54525055833

In [16]:
mae(y_test,y_pred)

109.41474722772277

In [17]:
R2=r2_score(y_test,y_pred)
R2

0.18299285317985048

In [18]:
R2_test=model.score(x_test,y_test)
R2_train=model.score(x_train,y_train)
Adj_R2= 1 - (1-R2)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)
Adj_R2

0.17247344785598595

In [19]:
model_lasso=Lasso(alpha=0.05)

model_lasso.fit(x_train, y_train)
print(f"{model_lasso.__class__.__name__}: Train -> {model_lasso.score(x_train, y_train)}, Test -> {model_lasso.score(x_test, y_test)}")

Lasso: Train -> 0.2968187162009327, Test -> 0.18273421095700626


In [20]:
model_ridge=Ridge(alpha=1000)
model_ridge.fit(x_train, y_train)
print(f"{model_ridge.__class__.__name__}: Train -> {model_ridge.score(x_train, y_train)}, Test -> {model_ridge.score(x_test, y_test)}")

Ridge: Train -> 0.23582253415644716, Test -> 0.10518651661100109


In [21]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

print("X_train has %d rows and %d columns"  %(x_train.shape[0],x_train.shape[1]))
print("-----------------------------------")
print("The coefficient of determination for the test data is R2=%.2f"
      %(model.score(x_test, y_test)))
print("The coefficient of determination for the train data is R2=%.2f"
      %(model.score(x_train, y_train)))


X_train has 2124 rows and 9 columns
-----------------------------------
The coefficient of determination for the test data is R2=0.72
The coefficient of determination for the train data is R2=1.00


In [22]:
scores=cross_val_score(model, x_train, y_train, cv=5)
print("Cross validation scores: ", scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Cross validation scores:  [-0.64144178  0.08158425  0.20922312 -2.82055573  0.19313265]
Score stats: -0.60 accuracy with a standard deviation of 1.16


In [23]:
model1 = DecisionTreeRegressor()
model2 = LinearRegression()
model3 = KNeighborsRegressor()

model_pipeline = [model1, model2, model3]
model_names = ['Regression Tree', 'Linear Regression', 'KNN']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, x_train, y_train, cv=5))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'Regression Tree': -0.5163834537684572, 'Linear Regression': 0.34663322567527477, 'KNN': 0.38362484175955347}
